In [47]:
# import
from IPython.display import HTML
import numpy as np
import bs4
import time
import operator
import socket
import re
import urllib

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from secret import *
import json

### Read the user data

In [2]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.user', 
                    sep='|', 
                    names=u_cols)
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Read the ratings

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
            'http://files.grouplens.org/datasets/movielens/ml-100k/u.data',
            sep='\t',
            names=r_cols)
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Data about the movies

In [4]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5))
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


### Get infomation about data

In [5]:
print(movies.dtypes)
print()
print(movies.describe())

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object

          movie_id  video_release_date
count  1682.000000                   0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


### Selecting data

In [6]:
print(users.head())
print(users['occupation'].head())
columns_you_want = ['occupation', 'sex']
print(users[columns_you_want].head())

   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213
0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object
   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F


### Filtering data

In [7]:
oldUsers = users[users.age > 25]
oldUsers.head()

,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


### Quizz
- Show users aged 40 and male <br/>
- Show the mean age of female programmers

In [8]:
# users aged 40 and male
users[(users.sex=='M') & (users.age==40)].head()

,user_id,age,sex,occupation,zip_code
18,19,40,M,librarian,02138
82,83,40,M,other,44133
115,116,40,M,healthcare,97232
199,200,40,M,programmer,93402
283,284,40,M,executive,92629


In [9]:
# the mean age of female programmers
users[(users.sex=='F') & (users.occupation=='programmer')].age.mean()

32.166666666666664

### Split-apply-combine
- splitting the data into groups based on some criteria
- applying a function to each group independently
- combining the results into data structure

http://i.imgur.com/yjNkiwL.png

in Pandas, 
- it's call groupby (split part)
- apply (count function) and combine (assign to other)


In [10]:
# split data
#grouped_data = ratings.groupby(ratings['user_id'])
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])

#count and combine
rating_per_user = grouped_data.count()
rating_per_user.head()

user_id
1    272
2     62
3     54
4     24
5    175
dtype: int64

### Quiz
- get the average rating per movie
- advanced: get the movie titles with the highest average rating

In [36]:
# get avg rating per movie
grouped_data = ratings['rating'].groupby(ratings['movie_id'])
avg_ratings = grouped_data.mean()
print(avg_ratings.head())

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64


In [37]:
# advanced: get the movie titles with the highest average rating
highest_avg_rating = avg_ratings.max()
good_movie_ids = avg_ratings[avg_ratings == highest_avg_rating].index
print('movie id')
print(good_movie_ids)
print('movie title')
print(movies[movies.movie_id.isin(good_movie_ids)].title)
print('Number of ratings per movies')
how_many_rating = rating_per_movie.count()
print(how_many_rating[avg_ratings == highest_avg_rating])

movie id
Int64Index([814, 1122, 1189, 1201, 1293, 1467, 1500, 1536, 1599, 1653], dtype='int64', name='movie_id')
movie title
813                         Great Day in Harlem, A (1994)
1121                       They Made Me a Criminal (1939)
1188                                   Prefontaine (1997)
1200           Marlene Dietrich: Shadow and Light (1996) 
1292                                      Star Kid (1997)
1466                 Saint of Fort Washington, The (1993)
1499                            Santa with Muscles (1996)
1535                                 Aiqing wansui (1994)
1598                        Someone Else's America (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object
Number of ratings per movies
movie_id
814     1
1122    1
1189    3
1201    1
1293    3
1467    2
1500    2
1536    1
1599    1
1653    1
dtype: int64


### Passing a Function

In [38]:
avg_ratings = grouped_data.apply(lambda f: f.mean())
avg_ratings.head()

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
dtype: float64

### Scraping with Python

In [44]:
url = 'http://www.crummy.com/software/BeautifulSoup'
source = urllib.request.urlopen(url).read()
print(source)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"\n"http://www.w3.org/TR/REC-html40/transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title>Beautiful Soup: We called him Tortoise because he taught us.</title>\n<link rev="made" href="mailto:leonardr@segfault.org">\n<link rel="stylesheet" type="text/css" href="/nb/themes/Default/nb.css">\n<meta name="Description" content="Beautiful Soup: a library designed for screen-scraping HTML and XML.">\n<meta name="generator" content="Markov Approximation 1.4 (module: leonardr)">\n<meta name="author" content="Leonard Richardson">\n</head>\n<body bgcolor="white" text="black" link="blue" vlink="660066" alink="red">\n<img align="right" src="10.1.jpg" width="250"><br />\n\n<p>You didn\'t write that awful page. You\'re just trying to get some\ndata out of it. Beautiful Soup is here to help. Since 2004, it\'s been\nsaving programmers hours or days of work on quick-turnaround\nscreen sc

### JSON
- JavaScript Object Notation
- human readable
- transmit attribute-value pairs

In [52]:
a= {'a': 1, 'b':2}
s = json.dumps(a)
a2 = json.loads(s)
print(s)
print(a2)

{"a": 1, "b": 2}
{'a': 1, 'b': 2}
